## # Direct usage of y values instead of one hot encoding.

Instead of providing a 1-hot-encoding of the y values during training (loos function required is categorical_crossentropy),
the y values can be directly used (loss function should be sparse_categorical_crossentropy)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
import pickle
warnings.simplefilter('ignore')

In [2]:
#from sklearn.datasets import fetch_mldata
#mnist = fetch_mldata('MNIST original')
#mnist

In [3]:
# %%time

#Alternate method of loading data - approx time 37s

X=np.loadtxt('../../Data/X.txt')
y=np.loadtxt('../../Data/y.txt')

In [4]:
#X, y = mnist["data"], mnist["target"]
X.shape, y.shape

((70000, 784), (70000,))

In [5]:
y=y.reshape(70000,1)

In [6]:
# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]
shuffle_index = np.random.permutation(60000)
X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]


In [7]:
X_train.shape, y_train.shape

((60000, 784), (60000, 1))

In [8]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [9]:
import keras
from keras.models import Sequential
from keras.layers import Dense


Using TensorFlow backend.


In [10]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
#classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu', input_dim = 11))
classifier.add(Dense(activation="relu", input_dim=784, units=300, kernel_initializer="uniform"))

# Adding the second hidden layer
#classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu'))
classifier.add(Dense(activation="relu", units=16, kernel_initializer="uniform"))

# Adding the third hidden layer
#classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu'))
classifier.add(Dense(activation="relu", units=16, kernel_initializer="uniform"))

# Adding the output layer
#classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))

classifier.add(Dense(activation="sigmoid", units=1, kernel_initializer="uniform"))

Instructions for updating:
Colocations handled automatically by placer.


In [11]:
# Compiling the ANN
#classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [12]:
y_train_5 = (y_train == 5)
y_test_5 = (y_test == 5)

In [13]:
pd.concat([pd.Series(y_train.ravel()), pd.Series(y_train_5.ravel())], axis=1).sample(n=10)

,0,1
55587,2.0,False
55877,1.0,False
16468,9.0,False
55395,6.0,False
13672,4.0,False
34616,0.0,False
7389,8.0,False
51257,3.0,False
37536,2.0,False
55343,3.0,False


In [14]:
filename = 'oneclass.sav'
try:
    classifier = pickle.load(open(filename, 'rb'))
    print('Classifier Loaded from file')
except:
    classifier.fit(X_train, y_train_5, batch_size = 128, epochs = 12)
    pickle.dump(classifier, open(filename, 'wb'))
    print('Model Saved in file ',filename )

Instructions for updating:
Use tf.cast instead.
Epoch 1/12
60000/60000 [==============================] - 3s 47us/step - loss: 0.0924 - acc: 0.9707
Epoch 2/12
60000/60000 [==============================] - 2s 35us/step - loss: 0.0231 - acc: 0.9925
Epoch 3/12
60000/60000 [==============================] - 2s 36us/step - loss: 0.0151 - acc: 0.9950
Epoch 4/12
60000/60000 [==============================] - 2s 37us/step - loss: 0.0102 - acc: 0.9968
Epoch 5/12
60000/60000 [==============================] - 2s 39us/step - loss: 0.0070 - acc: 0.9977
Epoch 6/12
60000/60000 [==============================] - 2s 38us/step - loss: 0.0064 - acc: 0.9980
Epoch 7/12
60000/60000 [==============================] - 2s 38us/step - loss: 0.0048 - acc: 0.9986
Epoch 8/12
60000/60000 [==============================] - 2s 38us/step - loss: 0.0041 - acc: 0.9987
Epoch 9/12
60000/60000 [==============================] - 2s 38us/step - loss: 0.0027 - acc: 0.9993
Epoch 10/12
60000/60000 [===========================

In [15]:
# Manual Fitting the ANN to the Training set

classifier.fit(X_train, y_train_5,  batch_size = 128, epochs = 12)

Epoch 1/12
60000/60000 [==============================] - 2s 34us/step - loss: 0.0026 - acc: 0.9992
Epoch 2/12
60000/60000 [==============================] - 2s 34us/step - loss: 0.0018 - acc: 0.9995
Epoch 3/12
60000/60000 [==============================] - 2s 37us/step - loss: 0.0012 - acc: 0.9997
Epoch 4/12
60000/60000 [==============================] - 2s 37us/step - loss: 0.0013 - acc: 0.9996
Epoch 5/12
60000/60000 [==============================] - 2s 37us/step - loss: 0.0030 - acc: 0.9992
Epoch 6/12
60000/60000 [==============================] - 2s 38us/step - loss: 0.0021 - acc: 0.9994
Epoch 7/12
60000/60000 [==============================] - 2s 38us/step - loss: 0.0021 - acc: 0.9995
Epoch 8/12
60000/60000 [==============================] - 2s 39us/step - loss: 0.0010 - acc: 0.9998
Epoch 9/12
60000/60000 [==============================] - 2s 39us/step - loss: 5.6856e-04 - acc: 0.9999
Epoch 10/12
60000/60000 [==============================] - 2s 41us/step - loss: 0.0015 - acc: 0.

In [16]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [17]:
y_pred[0:10]

array([[False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False]])

In [18]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test_5, y_pred)

In [19]:
cm

array([[9089,   19],
       [  31,  861]], dtype=int64)

In [20]:
# Assigning Variables for convinience
TN = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]
TP = cm[1][1]
print("TN=",TN, "TP=",TP, "FN=",FN,"FP=",FP)

TN= 9089 TP= 861 FN= 31 FP= 19


In [21]:
# acc=(TP+TN)/(TP+TN+FP+FN)
# print ('Accuracy ', acc)

accuracy = (TP+TN) / float(TP+TN+FN +FP)
print('Accuracy ', round(accuracy, 2))

sensitivity = TP / float(FN + TP)
print('Sensitivity or recall(5 Predictions):',round(sensitivity, 2))

specificity = TN / (TN + FP)
print('specificity(not 5 predictions)',round(specificity,2))

Accuracy  1.0
Sensitivity or recall(5 Predictions): 0.97
specificity(not 5 predictions) 1.0


# Observation :  Our sensitivity is 97% - compare to  94% for RandomForestclassifier 

# Multi Class Solution

In [31]:
# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]
shuffle_index = np.random.permutation(60000)
X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]


In [23]:
X_train.shape, y_train.shape

((60000, 784), (60000, 1))

In [24]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [25]:
# Initialising the ANN
classifier_mc = Sequential()

# Adding the input layer and the first hidden layer
#classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu', input_dim = 11))
classifier_mc.add(Dense(activation="relu", input_dim=784, units=300, kernel_initializer="uniform"))

# Adding the second hidden layer
#classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu'))
classifier_mc.add(Dense(activation="relu", units=16, kernel_initializer="uniform"))

# Adding the third hidden layer
#classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu'))
classifier_mc.add(Dense(activation="relu", units=16, kernel_initializer="uniform"))

# Adding the output layer
#classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))

classifier_mc.add(Dense(activation="softmax", units=10, kernel_initializer="uniform"))

In [47]:
# Compiling the ANN
# classifier_mc.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier_mc.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

# encode class values as integers
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
# convert integers to dummy variables (i.e. one hot encoded)
y_train = np_utils.to_categorical(encoded_Y)



In [36]:
y_train_full = np.zeros((y_train.shape[0], 10), dtype=np.int8)

In [40]:
y_train = y_train.astype(int).ravel()

In [42]:
for i in y_train:
    y_train_full[i, y_train[i]] = y_train[i]

In [44]:
y_train[0:10]

array([0, 1, 1, 2, 6, 2, 0, 1, 2, 9])

In [43]:
y_train_full[0:10]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 2, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 6, 0, 0, 0],
       [0, 0, 2, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 2, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 9]], dtype=int8)

In [51]:
filename = 'multiclass.sav'
try:
    classifier_mc = pickle.load(open(filename, 'rb'))
    print('Model Loaded')
except:
    classifier_mc.fit(X_train, y_train, batch_size = 128, epochs = 12)
    pickle.dump(classifier_mc, open(filename, 'wb'))
    print('Model Saved in file ',filename )

Model Loaded


In [52]:
y_pred_mc=classifier_mc.predict_classes(X_test)

In [53]:
y_pred_mc

array([0, 0, 0, ..., 9, 9, 9], dtype=int64)

In [54]:
matrix = confusion_matrix(y_test, y_pred_mc)

In [55]:
matrix.shape

(10, 10)

In [56]:
matrix

array([[ 964,    1,    2,    1,    1,    5,    4,    0,    1,    1],
       [   0, 1124,    2,    2,    0,    1,    1,    0,    5,    0],
       [   1,    2, 1010,    7,    0,    2,    4,    1,    5,    0],
       [   0,    0,    4,  992,    0,    4,    0,    2,    6,    2],
       [   0,    0,    2,    0,  954,    0,   13,    1,    2,   10],
       [   1,    0,    0,    8,    1,  872,    4,    1,    5,    0],
       [   3,    3,    2,    0,    2,    7,  939,    0,    2,    0],
       [   0,    6,   19,    5,    0,    0,    0,  988,    2,    8],
       [   3,    1,    2,    1,    1,    6,    3,    1,  951,    5],
       [   1,    2,    0,   13,   13,   10,    2,    5,    7,  956]],
      dtype=int64)

In [57]:
ok=0
for i in range(0,10):
    ok=ok+matrix[i,i]

print('Accuracy ', 100*ok/matrix.sum())

Accuracy  97.5


In [149]:
# Getting classification report
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred_mc))

             precision    recall  f1-score   support

        0.0       0.98      0.99      0.99       980
        1.0       0.99      0.99      0.99      1135
        2.0       0.97      0.97      0.97      1032
        3.0       0.97      0.98      0.97      1010
        4.0       0.98      0.97      0.98       982
        5.0       0.98      0.97      0.97       892
        6.0       0.98      0.98      0.98       958
        7.0       0.97      0.97      0.97      1028
        8.0       0.98      0.97      0.97       974
        9.0       0.97      0.97      0.97      1009

avg / total       0.98      0.98      0.98     10000



## Observation :  Our Recall, Precision, F1 is 98% - compare to 95% in RandomForestclassifier 